In [1]:
!pip install pandas
!pip install vllm
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 160.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 88.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB

In [2]:
import numpy as np

In [3]:
import pandas as pd
import vllm
from vllm import SamplingParams
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
def prompt(abstract, hyp) -> str:
    return f"Abstract: {abstract}\nHypothesis: {hyp}\nInstructions: Classify this abstract as either 0 (Not Relevant) or 1 (Relevant) for evaluating the provided hypothesis.\nScore: "

In [5]:
data = pd.read_csv("../../data/eval/protein_cooccurrence_abstracts.csv")

In [6]:
data

,Interaction Statement,Abstract,Interaction,Interaction Type
0,There exists an interaction between protein NM...,PMID: 123456. The intricate interplay between ...,1,Protein Modification
1,There exists an interaction between protein GN...,PMID: 123456. The present study focuses on the...,0,Different Cellular Compartments
2,There exists an interaction between protein SP...,PMID: 123456. The study focuses on the compart...,0,Different Cellular Compartments
3,There exists an interaction between protein GN...,PMID: 123456. The intricate interaction betwee...,1,Enzyme-Substrate Interaction
4,There exists an interaction between protein MA...,PMID: 123456. The present study focuses on the...,0,Non-Interacting Proteins
...,...,...,...,...
95,There exists an interaction between protein FN...,PMID: 123456. This study focuses on the indepe...,0,Separate Pathways
96,There exists an interaction between protein C3...,PMID: 123456. The present study focuses on the...,0,Non-Interacting Proteins
97,There exists an interaction between protein CC...,PMID: 123456. The intricate network of protein...,0,Non-Interacting Proteins
98,There exists an interaction between protein NM...,PMID: 123456. The focus of this study is to el...,0,Non-Interacting Proteins


In [7]:
data["text"] = data.apply(lambda row: prompt(row["Abstract"], row["Interaction Statement"]), axis = 1)

In [8]:
test_data = data.groupby("Interaction Type").apply(lambda group: group["text"].values, include_groups = True)
test_labels = data.groupby("Interaction Type").apply(lambda group: group["Interaction"].values, include_groups = True)

/tmp/ipykernel_461/3924126022.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_data = data.groupby("Interaction Type").apply(lambda group: group["text"].values, include_groups = True)
/tmp/ipykernel_461/3924126022.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_labels = data.groupby("Interaction Type").apply(lambda group: group["Interaction"].values, include_groups = True)


In [9]:
groups, texts = list(zip(*test_data.items()))
groups, labels = list(zip(*test_labels.items()))

In [10]:
model = vllm.LLM(model="lexu14/porpoise1", max_model_len=4000)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

INFO 09-18 20:38:18 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='lexu14/porpoise1', speculative_config=None, tokenizer='lexu14/porpoise1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=lexu14/porpoise1, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_output_proc=True)


tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-18 20:40:19 model_runner.py:1008] Loading model weights took 7.1183 GB
INFO 09-18 20:40:20 gpu_executor.py:122] # GPU blocks: 5519, # CPU blocks: 682
INFO 09-18 20:40:22 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-18 20:40:22 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-18 20:40:39 model_runner.py:1430] Graph capturing finished in 17 secs.


In [13]:
sampling_config = SamplingParams(temperature=0, top_p=0.95, top_k = 20, max_tokens = 1)

In [14]:
y_hats = []
for i in range(len(texts)):
    generated = model.generate(texts[i], sampling_params=sampling_config)
    pred = [int(output.outputs[0].text) for output in generated]
    y_hats.append(pred)

Processed prompts: 100%|██████████| 6/6 [00:00<00:00, 20.50it/s, est. speed input: 12029.67 toks/s, output: 20.50 toks/s]


In [15]:
groups

('Conflicting Evidence',
 'Different Cellular Compartments',
 'Direct Physical Interaction',
 'Enzyme-Substrate Interaction',
 'Functional Association',
 'Genetic Interaction',
 'No Direct Interaction',
 'Non-Interacting Proteins',
 'Parallel Functions',
 'Protein Complex Formation',
 'Protein Modification',
 'Regulatory Interaction',
 'Separate Pathways',
 'Signal Transduction Interaction')

In [16]:
labels

(array([1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1]),
 array([1, 1, 1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1]),
 array([1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]))

In [17]:
y_hats

[[1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1]]

In [18]:
rows = []

In [19]:
y = labels[0]
y_hat = y_hats[0]

In [20]:
accuracy_score, precision_score, recall_score, f1_score

(<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>,
 <function sklearn.metrics._classification.precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>,
 <function sklearn.metrics._classification.recall_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>,
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

In [21]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
y_hat

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [23]:
rows = []

In [24]:
for i in range(len(y_hats)):
    y = labels[i]
    y_hat = y_hats[i]
    accuracy = accuracy_score(y, y_hat)
    row = [groups[i], y_hat, y, accuracy]
    rows.append(row)

In [25]:
out_df = pd.DataFrame(rows, columns = ["group", "y_hat", "y", "accuracy"])

In [26]:
out_df

,group,y_hat,y,accuracy
0,Conflicting Evidence,"[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]",1.00
1,Different Cellular Compartments,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.95
2,Direct Physical Interaction,"[1, 1, 1]","[1, 1, 1]",1.00
3,Enzyme-Substrate Interaction,"[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]",1.00
4,Functional Association,"[1, 1, 1]","[1, 1, 1]",1.00
5,Genetic Interaction,"[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]",1.00
6,No Direct Interaction,"[1, 1, 1, 1]","[1, 1, 1, 1]",1.00
7,Non-Interacting Proteins,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.00
8,Parallel Functions,"[0, 1, 0, 1, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.60
9,Protein Complex Formation,"[1, 1, 1, 1]","[1, 1, 1, 1]",1.00


In [27]:
out_df.to_csv("results_protein.csv")